In [29]:
"""
准备数据集。
每个人单独标准化的数据，不需要再进行normalization。
每个人的数据前8训练后2测试。
"""

import joblib
import time
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE

from utils_logger import logger_init
import logging

from utils_model import form_params_grid
from utils_iFEMG_feature import df_save_csv

In [30]:
# 读数据
data_df = pd.read_csv(r"E:\Data\paper2\iFEMG\iFEMG_extension_abs_normed_all.csv", index_col = None, header = 0)
# data_df = pd.read_csv(r"E:\Data\paper2\iFEMG\iFEMG_extension_abs_normed_all.csv", index_col = None, header = 0)

print(data_df.shape)
data_df.head()

(363, 33)


,subject_info_height,subject_info_weight,subject_info_age,subject_info_gender,subject_info_name,subject_info_label,bicps_br_initial_pressure_ave,bicps_br_FMG,bicps_br_mav,bicps_br_rms,...,tricps_br_medial_mean_power_freq,tricps_br_lateral_initial_pressure_ave,tricps_br_lateral_FMG,tricps_br_lateral_mav,tricps_br_lateral_rms,tricps_br_lateral_wave_length,tricps_br_lateral_zero_crossing,tricps_br_lateral_slope_sign_change,tricps_br_lateral_mean_freq,tricps_br_lateral_mean_power_freq
0,172,80,23,1,hpy,0.0,0.39202,0.564416,-0.783479,-0.588296,...,-2.082373,1.337618,-0.433583,-0.337397,-0.341393,-0.299173,1.594339,0.471496,0.863801,0.682046
1,172,80,23,1,hpy,0.0,0.39202,-0.398311,4.001066,4.361124,...,-1.675564,1.337618,-0.688947,3.836401,3.234571,3.946198,-3.039795,-4.013581,-2.828318,-2.948408
2,172,80,23,1,hpy,0.0,0.39202,-0.221025,-0.701112,-0.411915,...,-0.065325,1.337618,-0.513747,-0.335000,-0.341471,-0.305690,0.872240,0.930294,0.641435,0.527620
3,172,80,23,1,hpy,0.0,0.39202,-0.258831,-0.811689,-0.592715,...,-0.145184,1.337618,-0.074921,-0.313549,-0.331923,-0.334710,-0.234919,-0.023807,0.120876,0.114766
4,172,80,23,1,hpy,0.0,0.39202,-0.078175,-0.295556,-0.304528,...,0.631359,1.337618,0.408739,-0.305507,-0.329471,-0.305711,-0.043220,0.020746,-0.038234,0.067408


In [31]:
"""
数据清洗 remove the data bellow:
    1. NaN value
    2. label is MVC
"""
# replace 'MVC' with NaN
data_df = data_df.replace('MVC', np.nan)
# delete NaN value
data_df = data_df.dropna(how = 'any')

print(data_df.shape)
data_df.dtypes

(363, 33)


subject_info_height                         int64
subject_info_weight                         int64
subject_info_age                            int64
subject_info_gender                         int64
subject_info_name                          object
subject_info_label                        float64
bicps_br_initial_pressure_ave             float64
bicps_br_FMG                              float64
bicps_br_mav                              float64
bicps_br_rms                              float64
bicps_br_wave_length                      float64
bicps_br_zero_crossing                    float64
bicps_br_slope_sign_change                float64
bicps_br_mean_freq                        float64
bicps_br_mean_power_freq                  float64
tricps_br_medial_initial_pressure_ave     float64
tricps_br_medial_FMG                      float64
tricps_br_medial_mav                      float64
tricps_br_medial_rms                      float64
tricps_br_medial_wave_length              float64


In [32]:
# 对数据中的个人信息进行标准化
# 注意请勿多次运行
subject_feature_columes = ['subject_info_height', 'subject_info_weight', 'subject_info_age']
data_df[subject_feature_columes] = preprocessing.StandardScaler().fit_transform(data_df[subject_feature_columes])
data_df

,subject_info_height,subject_info_weight,subject_info_age,subject_info_gender,subject_info_name,subject_info_label,bicps_br_initial_pressure_ave,bicps_br_FMG,bicps_br_mav,bicps_br_rms,...,tricps_br_medial_mean_power_freq,tricps_br_lateral_initial_pressure_ave,tricps_br_lateral_FMG,tricps_br_lateral_mav,tricps_br_lateral_rms,tricps_br_lateral_wave_length,tricps_br_lateral_zero_crossing,tricps_br_lateral_slope_sign_change,tricps_br_lateral_mean_freq,tricps_br_lateral_mean_power_freq
0,-1.52530,0.603985,0.260090,1,hpy,0.0,0.392020,0.564416,-0.783479,-0.588296,...,-2.082373,1.337618,-0.433583,-0.337397,-0.341393,-0.299173,1.594339,0.471496,0.863801,0.682046
1,-1.52530,0.603985,0.260090,1,hpy,0.0,0.392020,-0.398311,4.001066,4.361124,...,-1.675564,1.337618,-0.688947,3.836401,3.234571,3.946198,-3.039795,-4.013581,-2.828318,-2.948408
2,-1.52530,0.603985,0.260090,1,hpy,0.0,0.392020,-0.221025,-0.701112,-0.411915,...,-0.065325,1.337618,-0.513747,-0.335000,-0.341471,-0.305690,0.872240,0.930294,0.641435,0.527620
3,-1.52530,0.603985,0.260090,1,hpy,0.0,0.392020,-0.258831,-0.811689,-0.592715,...,-0.145184,1.337618,-0.074921,-0.313549,-0.331923,-0.334710,-0.234919,-0.023807,0.120876,0.114766
4,-1.52530,0.603985,0.260090,1,hpy,0.0,0.392020,-0.078175,-0.295556,-0.304528,...,0.631359,1.337618,0.408739,-0.305507,-0.329471,-0.305711,-0.043220,0.020746,-0.038234,0.067408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,-0.53481,-0.457742,0.846506,1,zx,1.0,1.373585,1.356243,-1.739720,-1.746915,...,-0.142765,-1.412972,-0.700391,0.827688,0.952529,1.023123,-0.204017,-0.910203,0.507252,0.380126
359,-0.53481,-0.457742,0.846506,1,zx,1.0,1.373585,1.343570,-0.436713,-0.490570,...,2.276380,-1.412972,-0.699043,0.823401,0.899010,0.901988,-0.344486,-0.874743,0.379317,0.176260
360,-0.53481,-0.457742,0.846506,1,zx,1.0,1.373585,1.383356,0.089016,-0.048551,...,-0.876041,-1.412972,-0.510883,1.924468,1.778597,0.868482,-2.448931,-1.110668,-1.180412,-1.262641
361,-0.53481,-0.457742,0.846506,1,zx,1.0,1.373585,1.406133,1.436197,1.040483,...,0.598261,-1.412972,-0.527897,0.911058,0.859205,1.082908,-0.216781,-1.196006,0.609825,0.453691


In [33]:
# 划分数据集，选择前80%作为训练集，后20%作为测试集
train_list = []
test_list = []

for (subject_info_name, subject_info_label), group in data_df.groupby(['subject_info_name', 'subject_info_label']):
    # 计算分割索引
    split_index = int(len(group) * 0.8)
    
    # 选取前80%作为训练集，后20%作为测试集
    train_list.append(group.iloc[:split_index])
    test_list.append(group.iloc[split_index:])

# 合并所有训练集和测试集
train_df = pd.concat(train_list).reset_index(drop=True)
test_df = pd.concat(test_list).reset_index(drop=True)
print(train_df.shape)
print(test_df.shape)

(276, 33)
(87, 33)


In [34]:
'''
最传统做法
不需要划分数据集，进行cv参数搜索
'''
columns_to_scale = ['subject_info_height',
                    'subject_info_weight',
                    'subject_info_age',
                    'bicps_br_initial_pressure_ave',
                    'bicps_br_FMG',
                    'bicps_br_mav',
                    'bicps_br_rms',
                    'bicps_br_wave_length',
                    'bicps_br_zero_crossing',
                    'bicps_br_slope_sign_change',
                    'bicps_br_mean_freq',
                    'bicps_br_mean_power_freq',
                    'tricps_br_medial_initial_pressure_ave',
                    'tricps_br_medial_FMG',
                    'tricps_br_medial_mav',
                    'tricps_br_medial_rms',
                    'tricps_br_medial_wave_length',
                    'tricps_br_medial_zero_crossing',
                    'tricps_br_medial_slope_sign_change',
                    'tricps_br_medial_mean_freq',
                    'tricps_br_medial_mean_power_freq',
                    'tricps_br_lateral_initial_pressure_ave',
                    'tricps_br_lateral_FMG',
                    'tricps_br_lateral_mav',
                    'tricps_br_lateral_rms',
                    'tricps_br_lateral_wave_length',
                    'tricps_br_lateral_zero_crossing',
                    'tricps_br_lateral_slope_sign_change',
                    'tricps_br_lateral_mean_freq',
                    'tricps_br_lateral_mean_power_freq']

train_data = train_df.loc[:, columns_to_scale].values
train_label = train_df.loc[:, 'subject_info_label'].values
test_data = test_df.loc[:, columns_to_scale].values
test_label = test_df.loc[:, 'subject_info_label'].values

"""
Grid searching best parameters of SVR model
"""
# 记录开始训练时间
start_time = time.perf_counter()

# 自动选择合适的参数
svr = GridSearchCV(SVR(kernel='rbf'), param_grid={"C": np.logspace(-3, 3, 7), "gamma": np.logspace(-3, 3, 7)}, n_jobs=-1)
svr.fit(train_data, train_label)
print(f"Best params: {svr.best_params_}")

end_time = time.perf_counter()
print(f"training time(min): {(end_time - start_time)/60}")

score_test = svr.score(test_data, test_label)
score_train = svr.score(train_data, train_label)
print(f"train score: {str(score_train)}")
print(f"test score: {str(score_test)}")
test_pre = svr.predict(test_data)
train_pre = svr.predict(train_data)
print(f"train mean squared error: {mean_squared_error(train_label, train_pre)}")
print(f"test mean squared error: {mean_squared_error(test_label, test_pre)}")


Best params: {'C': 1.0, 'gamma': 0.001}
training time(min): 0.03370571000001898
train score: 0.7370855822423926
test score: 0.689873191680033
train mean squared error: 0.0438156182187674
test mean squared error: 0.05164682809979105


In [35]:
test_err_df = pd.DataFrame({'y_true': test_label,
                            'y_predict': test_pre})

In [36]:
df_save_csv(test_err_df, r"E:\Data\paper2\肌力预测模型结果\test_err_extension_8020.csv")

File E:\Data\paper2\肌力预测模型结果\test_err_extension_8020.csv saved!
